In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\JAGADEESHNAIKPALTHYA\langgraph-end-to-end\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [9]:
llm.invoke("hi,how are you?")

AIMessage(content="As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 15, 'total_tokens': 47, 'completion_time': 0.058181818, 'prompt_time': 0.001884436, 'queue_time': 0.235046523, 'total_time': 0.060066254}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed0e7656-74de-41b8-9869-adb1541c4a84-0', usage_metadata={'input_tokens': 15, 'output_tokens': 32, 'total_tokens': 47})

### Pre-difined Tools

In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.run({"query": "langchan"}))

No good Wikipedia Search Result was found


In [15]:
from langchain_community.tools import YouTubeSearchTool
tool2=YouTubeSearchTool()
tool2.run("sunny savita")

"['https://www.youtube.com/watch?v=cydntJObPLI&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=xVGQrltjaRs&pp=ygUMc3Vubnkgc2F2aXRh']"

In [16]:
from langchain_community.tools import YouTubeSearchTool
tool2=YouTubeSearchTool()
tool2.run("carry minati")

"['https://www.youtube.com/watch?v=m9s1NQG3TNY&pp=ygUMY2FycnkgbWluYXRp', 'https://www.youtube.com/watch?v=i2xxhRTs5Uc&pp=ygUMY2FycnkgbWluYXRp']"

In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'title': 'Burning Man flooding: What happened to stranded festivalgoers?',
  'url': 'https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331',
  'content': '2:49\nAttendees look at a double rainbow over flooding on a desert plain at the Burning Man festival in Black Rock City, Nevada, September 1, 2023.\nJulie Jammot/AFP via Getty Images\nTens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend. [...] Camps are set on a muddy desert plain at the Burning Man festival in Black Rock, City, Nevada, Sept. 2, 2023.\nJulie Jammot/AFP via Getty Images\nWhat happened?\nBlack Rock Desert was faced with two to three months worth of rain in just a matter of hours on Friday, Sept. 1.\nOn average, the area gets only 0.2 inches of rain or less in September -- but festivalgoers were met with up to 1 inch in some areas, in a desert that gets only about 5 to 6 inches

### Custom tools

In [19]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [20]:
get_word_length.invoke('Hii')

3

In [21]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [23]:
@tool
def summisation(a: int, b: int) -> int:
    """Adding two numbers."""
    return a + b

In [24]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


### Agents

In [25]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [32]:
tool=load_tools(["wikipedia"],llm=llm)

In [27]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\JAGADEESHNAIKPALTHYA\AppData\Local\Temp\ipykernel_17544\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [34]:
agent.run('who is keerthi suresh')



> Entering new AgentExecutor chain...
Thought: I need to look up information about Keerthi Suresh on Wikipedia.
Action: wikipedia
Action Input: Keerthi Suresh
Observation: Page: Keerthy Suresh
Summary: Keerthy Suresh (born 17 October 1992) is an Indian actress who appears predominantly in  Tamil and Telugu films. Her accolades include one National Film Award, five SIIMA Awards and two Filmfare Awards South. Keerthy was placed in Forbes India's 30 Under 30 list of 2021. 
Keerthy is the daughter of film producer G. Suresh Kumar and actress Menaka. She began her career as a child actress in the early 2000s and returned to films after studying fashion design. She had her first lead role in the 2013 Malayalam film Geethaanjali, for which she won the SIIMA Award for Best Female Debut – Malayalam. After winning the SIIMA Award for Best Female Debut – Tamil for Idhu Enna Maayam (2015), Keerthy went on to star in successful films such as Ring Master (2014), Nenu Sailaja (2016), Rajinimurugan 

'Keerthy Suresh is an Indian actress who predominantly appears in Tamil and Telugu films. She is the recipient of multiple awards, including a National Film Award and five SIIMA Awards.'

In [35]:
agent.run("what is llama and who create this llm model?")



> Entering new AgentExecutor chain...
Thought: I need to look up "llama" on Wikipedia to find information about the LLM model.
Action: wikipedia
Action Input: llama (large language model)
Observation: Page: Llama (language model)
Summary: Llama (Large Language Model Meta AI, formerly stylized as LLaMA) is a family of large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 3.3, released in December 2024.
Llama models are trained at different parameter sizes, ranging between 1B and 405B. Initially  only a foundation model, starting with Llama 2, Meta AI released instruction fine-tuned versions alongside foundation models.
Model weights for the first version of Llama were only available to researchers on a case-by-case basis, under a non-commercial license. Unauthorized copies of the first model were shared via BitTorrent. Subsequent versions of Llama were made accessible outside academia and released under licenses that permitted some com

'Llama is a family of large language models created by Meta AI.'

In [36]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [37]:
search = TavilySearchResults()

In [38]:
search.invoke("what is the weather in SF")

[{'title': 'Weather in San Francisco',
  'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1743577117, 'localtime': '2025-04-01 23:58'}, 'current': {'last_updated_epoch': 1743576300, 'last_updated': '2025-04-01 23:45', 'temp_c': 10.6, 'temp_f': 51.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 18.8, 'wind_kph': 30.2, 'wind_degree': 309, 'wind_dir': 'NW', 'pressure_mb': 1014.0, 'pressure_in': 29.93, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 68, 'cloud': 25, 'feelslike_c': 7.3, 'feelslike_f': 45.2, 'windchill_c': 6.9, 'windchill_f': 44.5, 'heatindex_c': 9.7, 'heatindex_f': 49.4, 'dewpoint_c': 7.1, 'dewpoint_f': 44.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 27.2, 'gust_kph': 43.7}}",
  's

In [39]:
tools = [search]

In [40]:
from langchain import hub

In [41]:
prompt=hub.pull("hwchase17/openai-functions-agent")

In [42]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [43]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [44]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [46]:
agent_executor.invoke({"input": "What is the corrent GDP of India"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'What is the current GDP of India'}`


[{'title': 'India GDP - Worldometer', 'url': 'https://www.worldometers.info/gdp/india-gdp/', 'content': 'Population\nMore \nWGDPGDP by Country India GDP\n India GDP\nGDP nominal (2022)\n$3,385,090,000,000\n(Current US Dollars). Sources include: World Bank, United Nations.\nGDP growth rate (2022)\n7.00% \nNominal (current) Gross Domestic Product (GDP) of India is $3,385,090,000,000 (USD) as of 2022.\nReal GDP (constant, inflation adjusted) of India reached $2,432,020,000,000 in 2022. [...] GDP Growth Rate in 2022 was 7.00%, representing a change of 193,390,000,000 US$ over 2021, when Real GDP was $2,761,590,000,000.\nGDP per Capita in India (with a population of 1,425,423,212 people) was $2,073 in 2022, an increase of $120 from $1,953 in 2021; this represents a change of 6.2% in GDP per capita.\nCountryContinentWorld\nIndia GDP (Nominal, $USD) 2003-2022\

{'input': 'What is the corrent GDP of India', 'output': '\n'}

In [50]:

print(agent_executor.invoke({"input": "whats the weather in new york"}))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in new york'}`


[{'title': 'Weather in New York', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.7142, 'lon': -74.0064, 'tz_id': 'America/New_York', 'localtime_epoch': 1743584666, 'localtime': '2025-04-02 05:04'}, 'current': {'last_updated_epoch': 1743584400, 'last_updated': '2025-04-02 05:00', 'temp_c': 3.9, 'temp_f': 39.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 9.2, 'wind_kph': 14.8, 'wind_degree': 39, 'wind_dir': 'NE', 'pressure_mb': 1029.0, 'pressure_in': 30.38, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 35, 'cloud': 0, 'feelslike_c': 0.4, 'feelslike_f': 32.8, 'windchill_c': -1.8, 'windchill_f': 28.8, 'heatindex_c': 1.7, 'heatindex_f': 35.0, 'dewpoint_c': -2.4, 'dewpoint_f': 2